<a href="https://colab.research.google.com/github/DJCordhose/buch-machine-learning-notebooks/blob/master/kap10-openai-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMS - OpenAI GPT API Basics

* zum Ausführen ist ein OpenAI-Account notwendig: https://platform.openai.com/signup
* hier https://platform.openai.com/api-keys den eigenen Key erzeugen und eintragen
* Kosten
  * https://openai.com/pricing
  * https://platform.openai.com/usage
* Installation und Quickstart: https://platform.openai.com/docs/quickstart?context=python  


In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules # True if running in Google Colab.
IN_COLAB

True

In [2]:
if IN_COLAB:
    !pip install -q openai tiktoken

# Tokenizer

* Tokenizer zerlegen Texte in Zahlen
* diese Zahlen korrespondieren mit Token
* Token könnten ganze Worte, Wortteile, Ziffern, Sonderzeichen oder Satzzeiten sein

Links
* https://github.com/openai/tiktoken
* https://platform.openai.com/tokenizer

In [3]:
import tiktoken

# Dies ist der Tokenizer für GPT-4:
enc = tiktoken.encoding_for_model("gpt-4")
assert enc.decode(enc.encode("Hallo, Welt")) == "Hallo, Welt"
enc.encode("Hallo, Welt!0")

[79178, 11, 46066, 0, 15]

# OpenAI Client

In [4]:
# TODO: den Key eintragen
OPENAI_API_KEY = '...'


assert OPENAI_API_KEY != '...', "Bitte tragen Sie Ihren OpenAI API Key ein!"

In [5]:
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_API_KEY
)
models = client.models.list()

In [6]:
set([model.owned_by for model in models])

{'openai', 'openai-internal', 'system'}

In [7]:
set([model.id for model in models if model.owned_by == 'openai'])

{'gpt-3.5-turbo',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-4',
 'gpt-4-0613'}

In [8]:
set([model.id for model in models if model.owned_by == 'system'])

{'babbage-002',
 'dall-e-2',
 'dall-e-3',
 'davinci-002',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-1106',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-4-0125-preview',
 'gpt-4-1106-preview',
 'gpt-4-1106-vision-preview',
 'gpt-4-turbo',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-turbo-preview',
 'gpt-4-vision-preview',
 'gpt-4o',
 'gpt-4o-2024-05-13',
 'text-embedding-3-large',
 'text-embedding-3-small',
 'tts-1-1106',
 'tts-1-hd',
 'tts-1-hd-1106'}

In [9]:
set([model.id for model in models if model.owned_by == 'openai-internal'])

{'gpt-3.5-turbo-16k', 'text-embedding-ada-002', 'tts-1', 'whisper-1'}

In [10]:
# https://platform.openai.com/docs/api-reference/chat/create
completion = client.chat.completions.create(
  # model="gpt-3.5-turbo",
  model="gpt-4o",

  messages=[
    {"role": "system", "content": "Du bist ein Experte im Bereich Machine Learning. Deine Antworten sind kurz und gut."},
    {"role": "user", "content": "Warum sind große Sprachmodelle so mächtig?"},
  ],
  logprobs=True,
  top_logprobs=3
)
completion.choices[0].message.content

'Große Sprachmodelle sind mächtig aufgrund:\n\n1. **Skalierung**: Viele Parameter, die komplexe Muster erfassen.\n2. **Datenvielfalt**: Training auf enormen, vielfältigen Datensätzen.\n3. **Architektur**: Transformer-Architektur, die effektiv Kontext versteht.\n4. **Feinabstimmung**: Anpassung an spezifische Aufgaben durch Fine-Tuning.\n5. **Generalisierung**: Fähigkeit, Wissen zu transferieren und vielseitige Probleme zu lösen.'

In [11]:
for token in completion.choices[0].logprobs.content:
  print("---")
  print(token.token)
  for top_token in token.top_logprobs:
    print(f"{top_token.token}: {top_token.logprob}")

---
Gro
Gro: -0.000117370815
Gr: -9.625117
G: -11.125117
---
ße
ße: -1.6524515e-05
ß: -11.125016
ßen: -13.375016
---
 Sprach
 Sprach: -6.704273e-07
 Modelle: -14.750001
 sprach: -16.125
---
model
model: -2.577686e-06
module: -13.750003
modell: -13.875003
---
le
le: 0.0
lek: -21.5
lem: -22.125
---
 sind
 sind: -0.024960347
 wie: -4.1499605
,: -4.7749605
---
 mä
 mä: -0.10221221
 so: -2.3522122
 deshalb: -7.2272124
---
chtig
chtig: -5.5122365e-07
cht: -14.750001
chtigt: -16.625
---
 aufgrund
,: -0.25334388
 aufgrund: -2.1283438
 wegen: -2.3783438
---
:


 ihrer: -0.019424668
 mehr: -5.0194244
 von: -5.2694244
---
1
1: -3.5716304e-05
-: -10.250035
 : -16.125036
---
.
.: -9.0883464e-07
): -14.125001
-: -16.75
---
 **
 **: -0.0006212961
 ***: -8.500621
 Umfang: -8.500621
---
Sk
D: -0.7418427
Sk: -1.1168427
Mass: -2.7418427
---
al
al: -0.021226155
alen: -4.146226
ala: -5.271226
---
ierung
ierung: -0.07180578
ier: -2.8218057
ene: -5.196806
---
**
**: -0.15021011
:**: -2.6502101
 der: -3.15021